In [108]:
import os
from pybfe.client.session import Session
os.environ['BFE_SSL_CERT'] = "./nexbfe.crt"

In [109]:
BFE_HOST = "batfish.nexariacloud.com"
BFE_PORT = 443

In [116]:
NETWORK_NAME="AcmeCorp"
SNAPSHOT_NAME="baseline"
SNAPSHOT_DIR="../bgp_configs/changed_bgp/" 

In [117]:
#create session
bf = Session(host=BFE_HOST, port=BFE_PORT)
bf

In [118]:
#set network
'''
networks = bf.list_networks()
if NETWORK_NAME in networks:
    bf.delete_network(NETWORK_NAME)

bf.set_network(NETWORK_NAME)
'''
bf.set_network(NETWORK_NAME)

'AcmeCorp'

In [133]:
#init snapshot
bf.init_snapshot(name=SNAPSHOT_NAME, overwrite=True)

TypeError: init_snapshot() missing 1 required positional argument: 'upload'

In [119]:
networks = bf.list_networks()
print(networks)
snapshots = bf.list_snapshots()
print(snapshots)

['Example Network', 'AcmeCorp', 'bgp_test_lab']
['routes_test3', 'routes_test2', 'routes_test', 'duplicate_bgp_test', 'changed', 'baseline1', 'baseline']


In [44]:
#help(bf.delete_snapshot)
snapshots = bf.list_snapshots()
for snapshot in snapshots:
    if not snapshot.startswith('baseline'):
        bf.delete_snapshot(snapshot)

In [103]:
def clearAll():
    '''Deletes all networks except the default'''
    networks = bf.list_networks()
    for network in networks:
        if network.startswith("Example"):
            #print(network)
            bf.set_network(network)
        else:
            bf.delete_network(network)

In [104]:
clearAll()

In [100]:
#bf.set_network('Example Network')
bf.delete_network('bgp_test_lab')

ValueError: Network bgp_test_lab was not found.

In [105]:
#bf.delete_snapshot('baseline')
bf.network
#bf.list_network()

'Example Network'

In [120]:
REF_SNAPSHOT = ''
NEW_SNAPSHOT = ''
snapshots = bf.list_snapshots()
#case when tere are just 2 snapshots, where reference snapshot is called baseline and is manually uploaded
if len(snapshots) == 2:
    for snapshot in snapshots:
        if snapshot.startswith('baseline'):
            REF_SNAPSHOT = snapshot
        else: NEW_SNAPSHOT = snapshot
else:
    for snapshot in snapshots:
        if snapshot.startswith('baseline'):
            REF_SNAPSHOT = snapshot
            break
    NEW_SNAPSHOT = snapshots[0]

In [121]:
print("REF_SNAPSHOT :: ", REF_SNAPSHOT)
print("NEW_SNAPSHOT :: ", NEW_SNAPSHOT)

REF_SNAPSHOT ::  baseline1
NEW_SNAPSHOT ::  routes_test3


In [125]:
bf.init_snapshot(NEW_SNAPSHOT)

'routes_test3'

In [129]:
help(bf.init_snapshot)

Help on method init_snapshot in module pybfe.client.session:

init_snapshot(upload: str, name: Union[str, NoneType] = None, overwrite: bool = False, extra_args: Union[Dict[str, Any], NoneType] = None) -> str method of pybfe.client.session.Session instance
    Initialize a new snapshot.
    
    :param upload: path to the snapshot zip or directory
    :type upload: str
    :param name: name of the snapshot to initialize
    :type name: str
    :param overwrite: whether or not to overwrite an existing snapshot with the
       same name
    :type overwrite: bool
    :param extra_args: extra arguments to be passed to the parse command
    :type extra_args: dict
    
    :return: name of initialized snapshot
    :rtype: str



In [122]:
from intentionet.bfe.proto import api_gateway_pb2 as api

In [127]:
def get_compare_metadata_results(bf: Session, snapshot_name: str, reference_snapshot_name: str):
    """
    Gets snapshot comparison results.
    
    Returns a map from comparison type to either a count of changes or whether that type has changed.
    """
    resp = bf._api_gw.GetSnapshotComparisonMetadata(
        api.GetSnapshotComparisonMetadataRequest(
            network_name=bf.network,
            snapshot_name=snapshot_name,
            reference_snapshot_name=reference_snapshot_name
        )
    )
    
    return resp    

In [128]:
compare_response = get_compare_metadata_results(bf, NEW_SNAPSHOT, REF_SNAPSHOT)
compare_response

uninitialized: true

In [52]:
def get_compare_config_summary_results(bf: Session, snapshot_name: str, reference_snapshot_name: str):
    
    """
    Gets snapshot comparison results.
    
    Returns a map from comparison type to either a count of changes or whether that type has changed.
    """
    resp = bf._api_gw.GetSnapshotComparisonDevices(
        api.GetSnapshotComparisonDevicesRequest(
            network_name=bf.network,
            snapshot_name=snapshot_name,
            reference_snapshot_name=reference_snapshot_name
        )
    )
    
    return resp

In [57]:
compare_response = get_compare_config_summary_results(bf, NEW_SNAPSHOT, REF_SNAPSHOT)
compare_response

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNIMPLEMENTED
	details = "Method not implemented!"
	debug_error_string = "{"created":"@1622312463.720000000","description":"Error received from peer ipv4:54.197.65.81:443","file":"src/core/lib/surface/call.cc","file_line":1068,"grpc_message":"Method not implemented!","grpc_status":12}"
>

In [48]:
help(api.CreatePolicyRequest)

Help on class CreatePolicyRequest in module intentionet.bfe.proto.api_gateway_pb2:

class CreatePolicyRequest(google.protobuf.pyext._message.CMessage, google.protobuf.message.Message)
 |  A ProtocolMessage
 |  
 |  Method resolution order:
 |      CreatePolicyRequest
 |      google.protobuf.pyext._message.CMessage
 |      google.protobuf.message.Message
 |      builtins.object
 |  
 |  Data descriptors defined here:
 |  
 |  credentials
 |      Field com.intentionet.bfe.api_gateway.api.CreatePolicyRequest.credentials
 |  
 |  network_name
 |      Field com.intentionet.bfe.api_gateway.api.CreatePolicyRequest.network_name
 |  
 |  policy
 |      Field com.intentionet.bfe.api_gateway.api.CreatePolicyRequest.policy
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  DESCRIPTOR = <google.protobuf.pyext._message.MessageDescriptor object>
 |  
 |  ------------------------------------------------------------------

In [41]:
dir(bf._api_gw)

['ActivateLicense',
 'AddAwsAccount',
 'CreateAdHocAssertion',
 'CreateChangeReview',
 'CreateChangeReviewStep',
 'CreateNetwork',
 'CreatePolicy',
 'DeleteAwsAccount',
 'DeleteChangeReviewStep',
 'DeleteNetwork',
 'DeletePolicy',
 'DeleteQuestion',
 'DeleteReferenceBook',
 'DeleteSnapshot',
 'DeleteTopologyAggregates',
 'DeleteTopologyPositions',
 'DeleteTopologyRoots',
 'ForkFromMasterSnapshot',
 'ForkSnapshot',
 'GetAdHocAssertion',
 'GetAdHocAssertionResult',
 'GetAssertionInputJson',
 'GetAwsAccountsStatuses',
 'GetChangeReviewChange',
 'GetChangeReviewMetadata',
 'GetChangeReviewStep',
 'GetChangeReviewStepFromJson',
 'GetChangeReviewStepJson',
 'GetCloudFetchSettings',
 'GetCloudFetchStatus',
 'GetDataRetentionPolicy',
 'GetDebugSnapshots',
 'GetInsight',
 'GetInsightResult',
 'GetLicenseMetadata',
 'GetLicenseUser',
 'GetNetworkAggregates',
 'GetNetworkMetadata',
 'GetPolicy',
 'GetPolicyResult',
 'GetQuestion',
 'GetQuestions',
 'GetReferenceBook',
 'GetReferenceLibrary',
 'Ge

In [50]:
dir(api)

['ACCOUNT_STATUS_ERROR',
 'ACCOUNT_STATUS_OK',
 'ACCOUNT_STATUS_WAITING',
 'ACCOUNT_STATUS_WARNING',
 'AWS',
 'AWS_US_GOV',
 'ActivateLicenseRequest',
 'ActivateLicenseResponse',
 'AddAwsAccountRequest',
 'AddAwsAccountResponse',
 'AwsAccount',
 'AwsPartition',
 'CloudFetchAccountStatus',
 'CloudFetchAccountStatusCode',
 'CloudFetchAwsSettings',
 'CloudFetchProviderStatus',
 'CloudFetchProviderStatusCode',
 'CloudFetchSettings',
 'ColumnFilter',
 'ColumnSortOption',
 'CreateAdHocAssertionRequest',
 'CreateAdHocAssertionResponse',
 'CreateNetworkRequest',
 'CreateNetworkResponse',
 'CreatePolicyRequest',
 'CreatePolicyResponse',
 'Credentials',
 'DESCRIPTOR',
 'DataRetentionPolicy',
 'DeleteAwsAccountRequest',
 'DeleteAwsAccountResponse',
 'DeleteNetworkRequest',
 'DeleteNetworkResponse',
 'DeletePolicyRequest',
 'DeletePolicyResponse',
 'DeleteQuestionRequest',
 'DeleteQuestionResponse',
 'DeleteReferenceBookRequest',
 'DeleteReferenceBookResponse',
 'DeleteSnapshotRequest',
 'DeleteSn

In [70]:
bf.init_snapshot('../bgp_configs/reference_bgp', 'baseline', overwrite=True)

'baseline'

In [49]:

def policy_status_to_string(status):
    if status == policies_api.POLICY_STATUS_UNKNOWN:
        return "UNKNOWN"
    elif status == policies_api.POLICY_STATUS_FAIL:
        return "FAIL"
    elif status == policies_api.POLICY_STATUS_PASS:
        return "PASS"
    elif status == policies_api.POLICY_STATUS_PASS_WARN:
        return "WARN"
    else:
        raise RuntimeError("Unknown policy status {}".format(status))


def get_policy_results(bf: Session):
    """
    Get policy evaluation results for a snapshot.
    
    Returns a map from policy title to status
    """
    response = bf._api_gw.ListPolicyResultsMetadata(
        api.ListPolicyResultsMetadataRequest(
            network_name=bf.network, snapshot_name=bf.snapshot
        )
    )
    status = {}
    for result in response.metadata:
        policy_response = bf._api_gw.GetPolicy(
            api.GetPolicyRequest(
                network_name=bf.network, policy_id=result.policy_id
            )
        )
        status[policy_response.policy.input.title] = policy_status_to_string(result.status)

    return status

get_policy_results(bf)

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "Exception calling application: Snapshot  does not exist"
	debug_error_string = "{"created":"@1622379304.568000000","description":"Error received from peer ipv4:54.197.65.81:443","file":"src/core/lib/surface/call.cc","file_line":1068,"grpc_message":"Exception calling application: Snapshot  does not exist","grpc_status":2}"
>

In [50]:
bf.network

'bgp_test_lab'

In [54]:
bf.snapshot

'baseline'

In [55]:
from intentionet.bfe.proto import api_gateway_pb2 as api
from intentionet.bfe.proto import policies_api_pb2 as policies_api

In [61]:
request = api.ListPolicyResultsMetadataRequest(network_name=bf.network, snapshot_name=bf.snapshot)
print(request)

network_name: "bgp_test_lab"
snapshot_name: "baseline"



In [66]:
response = bf._api_gw.ListPolicyResultsMetadata(request)
print(response.metadata)

[]


In [71]:
bf.network

'bgp_test_lab'

In [72]:
bf.delete_network('bgp_test_lab')